# Question 1)

## a)

Given that 
$$
dr_t = \alpha(r_t,t)dt + \beta(r_t,t)dW_t
$$

We have that 
$$
dC_t = \frac{\partial C}{\partial t}dt + \frac{\partial C}{\partial r} dr_t + \frac{1}{2}\frac{\partial^2 C}{\partial r^2}d<r>_t
$$

$$
= \left[ \frac{\partial C}{\partial t} + \alpha \frac{\partial C}{\partial r} + \frac{1}{2}\beta^2\frac{\partial^2 C}{\partial r^2}\right]dt + \beta \frac{\partial C}{\partial r}dW_t
$$

Shorting $\beta\frac{\partial C}{\partial t}$ to cancel the $dW_t$ term. 

$$
\frac{\partial C}{\partial t} + \alpha \frac{\partial C}{\partial r} + \frac{1}{2}\beta^2\frac{\partial^2 C}{\partial r^2} = r_t\left[C_t -\beta \frac{\partial C}{\partial r}\right]
$$

$$
\frac{\partial C}{\partial t} + \left(\alpha(r_t,t)+r_t\cdot\beta(r_t,t)\right)\frac{\partial C}{\partial r} + \frac{1}{2}\beta(r_t,t)^2\frac{\partial^2 C}{\partial r^2} - r_tC_t = 0 
$$

## b)

We are now given that $\alpha(r_t,t) = \kappa(\theta - r_t)$ and $\beta(r_t,t) = \sigma$, which leads to the following PDE:

$$
\frac{\partial C}{\partial t} + \left[\kappa(\theta - r_t) +r_t\sigma \right]\frac{\partial C}{\partial r} + \frac{\sigma^2}{2}\frac{\partial^2 C}{\partial r^2} - r_tC = 0
$$

Define $\nu = \kappa(\theta - r_t) +r_t\sigma $ and use the central-difference explicit finite difference scheme to the PDE.

$$
\frac{\partial C}{\partial t} + \nu\frac{\partial C}{\partial r} + \frac{\sigma^2}{2}\frac{\partial^2 C}{\partial r^2} - r_tC = 0
$$

In [1]:
import numpy as np

In [2]:
class Dynamics:
    pass

hw3dynamics=Dynamics()
hw3dynamics.kappa = 3
hw3dynamics.theta = 0.05
hw3dynamics.sigma = 0.03

class Contract:
    pass

hw3contract=Contract()
hw3contract.T = 5

class FD:
    pass

hw3FD=FD()
hw3FD.rMax=0.35
hw3FD.rMin=-0.25
#|v| < 1 so deltar <= sigma^2/|v| and |v| <= 0.85 so deltar <= sigma^2 should work  
hw3FD.deltar=hw3dynamics.sigma**2
#delta t <= 1/sigma**2 * (deltar**2) = sigma**2 then take the nearest lowest value 5/x = integer
hw3FD.deltat=0.0005
hw3FD.useUpwind=False

hw3FD.deltar, hw3FD.deltat

(0.0009, 0.0005)

In [3]:
# You complete the coding of this function

def pricer_bond_vasicek_explicitFD(contract,dynamics,fd):
# returns array of all initial short rates,
# and the corresponding array of zero-coupon
# T-maturity bond prices

    T = contract.T
    kappa, theta, sigma = dynamics.kappa, dynamics.theta, dynamics.sigma    
    rMax, rMin, deltar, deltat = fd.rMax, fd.rMin, fd.deltar, fd.deltat    
    N=round(T/deltat)
    if abs(N-T/deltat) > 1e-12:
        raise ValueError("Bad delta t")
        
    r=np.arange(rMax,rMin-deltar/2,-deltar)   #I'm making the FIRST indices of the array correspond to HIGH levels of r
    bondprice=np.ones(np.size(r))
    
    v = kappa*(theta-r)+r*sigma
    
    if fd.useUpwind:
        qu=    np.where((v-r*sigma)>=0,1,0) * (v*deltat/deltar +(sigma**2*deltat)/(2*deltar**2)) + np.where((v-r*sigma)<0,1,0)*0.5*((sigma**2*deltat)/(deltar**2))
        qd=    np.where((v-r*sigma)>=0,1,0) * (sigma**2*deltat)/(2*deltar**2) + np.where((v-r*sigma)<0,1,0)* 0.5*((sigma**2*deltat)/(deltar**2)-(2*v*deltat)/(deltar))
        qm=    1-qu-qd
    else:
        qu=    0.5*((sigma**2*deltat)/(deltar**2)+(v*deltat)/(deltar))
        qd=    0.5*((sigma**2*deltat)/(deltar**2)-(v*deltat)/(deltar))
        qm=    1-qu-qd

    for t in np.arange(N-1,-1,-1)*deltat:
        bondprice=1/(1+r*deltat)*(qd*np.roll(bondprice,-1)+qm*bondprice+qu*np.roll(bondprice,1))
        
        # It is not obvious in this case, 
        # what boundary conditions to impose at the top and bottom
        # so let us impose "linearity" boundary conditions
        bondprice[0]=2*bondprice[1]-bondprice[2]
        bondprice[-1]=2*bondprice[-2]-bondprice[-3]
        
    return (r, bondprice)

In [4]:
(r, bondprice) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)
displayrows=np.logical_and(r<0.15+hw3FD.deltar/2, r>0.0-hw3FD.deltar/2)

In [5]:
print(np.stack((r, bondprice),1)[displayrows])

[[ 0.1502  0.7514]
 [ 0.1493  0.7516]
 [ 0.1484  0.7518]
 [ 0.1475  0.7521]
 [ 0.1466  0.7523]
 [ 0.1457  0.7525]
 [ 0.1448  0.7527]
 [ 0.1439  0.753 ]
 [ 0.143   0.7532]
 [ 0.1421  0.7534]
 [ 0.1412  0.7536]
 [ 0.1403  0.7539]
 [ 0.1394  0.7541]
 [ 0.1385  0.7543]
 [ 0.1376  0.7546]
 [ 0.1367  0.7548]
 [ 0.1358  0.755 ]
 [ 0.1349  0.7552]
 [ 0.134   0.7555]
 [ 0.1331  0.7557]
 [ 0.1322  0.7559]
 [ 0.1313  0.7562]
 [ 0.1304  0.7564]
 [ 0.1295  0.7566]
 [ 0.1286  0.7569]
 [ 0.1277  0.7571]
 [ 0.1268  0.7573]
 [ 0.1259  0.7575]
 [ 0.125   0.7578]
 [ 0.1241  0.758 ]
 [ 0.1232  0.7582]
 [ 0.1223  0.7585]
 [ 0.1214  0.7587]
 [ 0.1205  0.7589]
 [ 0.1196  0.7591]
 [ 0.1187  0.7594]
 [ 0.1178  0.7596]
 [ 0.1169  0.7598]
 [ 0.116   0.7601]
 [ 0.1151  0.7603]
 [ 0.1142  0.7605]
 [ 0.1133  0.7608]
 [ 0.1124  0.761 ]
 [ 0.1115  0.7612]
 [ 0.1106  0.7615]
 [ 0.1097  0.7617]
 [ 0.1088  0.7619]
 [ 0.1079  0.7621]
 [ 0.107   0.7624]
 [ 0.1061  0.7626]
 [ 0.1052  0.7628]
 [ 0.1043  0.7631]
 [ 0.1034  0

## c)

Define the event $\mathcal{G}:=\left\{\kappa(\theta - r_j) \geq 0\right\}$ and $\mathcal{X}(x)$ be the characteristic function or Identicator function, we can then represent the discretization of the PDE according to the following rule:

$$
\frac{\partial C}{\partial t} \approx \frac{C_{n+1}^j - C_n^j}{\Delta t}
$$
$$
\frac{\partial C}{\partial r} \approx \left(\frac{C_{n+1}^{j+1} - C_{n+1}^j}{\Delta r}\right) \cdot \mathcal{X}_{\{\mathcal{G}\}} + \left(\frac{C_{n+1}^{j} - C_{n+1}^{j-1}}{\Delta r}\right) \cdot \mathcal{X}_{\{\not \mathcal{G}\}}
$$
$$
\frac{\partial^2 C}{\partial r^2} \approx \frac{C_{n+1}^{j+1}-2C_{n+1}^j+C_{n+1}^{j-1}}{(\Delta r)^2}
$$

For events in $\mathcal{G}$:

$$
rC_n^j \approx \frac{C_{n+1}^j - C_n^j}{\Delta t} + \nu \frac{C_{n+1}^{j+1} - C_{n+1}^j}{\Delta r} + \frac{1}{2}\sigma^2 \frac{C_{n+1}^{j+1}-2C_{n+1}^j+C_{n+1}^{j-1}}{(\Delta r)^2}
$$
leads to 

$$
C_n^j \approx \frac{1}{1+r\Delta t}\left(
C_{n+1}^{j+1}\left(\frac{\nu \Delta t}{\Delta r} + \frac{\sigma^2 \Delta t}{2(\Delta r)^2}\right)+
C_{n+1}^{j}\left(1 -\frac{\nu \Delta t}{\Delta r} - \frac{\sigma^2 \Delta t}{(\Delta r)^2}\right)+
C_{n+1}^{j-1}\left(\frac{\sigma^2 \Delta t}{2(\Delta r)^2}\right)
\right)
$$

Given that 
$$
C_n^j \approx \frac{1}{1+r\Delta t}\left(C_{n+1}^{j+1} \cdot q_u + C_{n+1}^{j} \cdot q_m + C_{n+1}^{j-1} \cdot q_d\right)
$$

$$
q_u = \frac{\nu \Delta t}{\Delta r} + \frac{\sigma^2 \Delta t}{2(\Delta r)^2} \\
q_m = 1 -\frac{\nu \Delta t}{\Delta r} - \frac{\sigma^2 \Delta t}{(\Delta r)^2} \\
q_d = \frac{\sigma^2 \Delta t}{2(\Delta r)^2}
$$

For events not in $\mathcal{G}$:

$$
rC_n^j \approx \frac{C_{n+1}^j - C_n^j}{\Delta t} + \nu \frac{C_{n+1}^{j} - C_{n+1}^{j-1}}{\Delta r} + \frac{1}{2}\sigma^2 \frac{C_{n+1}^{j+1}-2C_{n+1}^j+C_{n+1}^{j-1}}{(\Delta r)^2}
$$
leads to 

$$
C_n^j \approx \frac{1}{1+r\Delta t}\left(
C_{n+1}^{j+1}\left(\frac{\sigma^2 \Delta t}{2(\Delta r)^2}\right)+
C_{n+1}^{j}\left(1 +\frac{\nu \Delta t}{\Delta r} - \frac{\sigma^2 \Delta t}{(\Delta r)^2}\right)+
C_{n+1}^{j-1}\left(\frac{\sigma^2 \Delta t}{2(\Delta r)^2}-\frac{\nu \Delta t}{\Delta r}\right)
\right)
$$

Given that 
$$
C_n^j \approx \frac{1}{1+r\Delta t}\left(C_{n+1}^{j+1} \cdot q_u + C_{n+1}^{j} \cdot q_m + C_{n+1}^{j-1} \cdot q_d\right)
$$

$$
q_u = \frac{\sigma^2 \Delta t}{2(\Delta r)^2} \\
q_m = 1 +\frac{\nu \Delta t}{\Delta r} - \frac{\sigma^2 \Delta t}{(\Delta r)^2} \\
q_d = \frac{\sigma^2 \Delta t}{2(\Delta r)^2} - \frac{\nu \Delta t}{\Delta r}
$$

In [6]:
hw3FD.useUpwind=True
(r2, bondprice2) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)
displayrows=np.logical_and(r2<0.15+hw3FD.deltar/2, r>0.0-hw3FD.deltar/2)
print(np.stack((r2, bondprice2),1)[displayrows])

[[ 0.1502  0.7514]
 [ 0.1493  0.7516]
 [ 0.1484  0.7518]
 [ 0.1475  0.7521]
 [ 0.1466  0.7523]
 [ 0.1457  0.7525]
 [ 0.1448  0.7527]
 [ 0.1439  0.753 ]
 [ 0.143   0.7532]
 [ 0.1421  0.7534]
 [ 0.1412  0.7537]
 [ 0.1403  0.7539]
 [ 0.1394  0.7541]
 [ 0.1385  0.7543]
 [ 0.1376  0.7546]
 [ 0.1367  0.7548]
 [ 0.1358  0.755 ]
 [ 0.1349  0.7553]
 [ 0.134   0.7555]
 [ 0.1331  0.7557]
 [ 0.1322  0.7559]
 [ 0.1313  0.7562]
 [ 0.1304  0.7564]
 [ 0.1295  0.7566]
 [ 0.1286  0.7569]
 [ 0.1277  0.7571]
 [ 0.1268  0.7573]
 [ 0.1259  0.7575]
 [ 0.125   0.7578]
 [ 0.1241  0.758 ]
 [ 0.1232  0.7582]
 [ 0.1223  0.7585]
 [ 0.1214  0.7587]
 [ 0.1205  0.7589]
 [ 0.1196  0.7592]
 [ 0.1187  0.7594]
 [ 0.1178  0.7596]
 [ 0.1169  0.7598]
 [ 0.116   0.7601]
 [ 0.1151  0.7603]
 [ 0.1142  0.7605]
 [ 0.1133  0.7608]
 [ 0.1124  0.761 ]
 [ 0.1115  0.7612]
 [ 0.1106  0.7615]
 [ 0.1097  0.7617]
 [ 0.1088  0.7619]
 [ 0.1079  0.7622]
 [ 0.107   0.7624]
 [ 0.1061  0.7626]
 [ 0.1052  0.7628]
 [ 0.1043  0.7631]
 [ 0.1034  0

## d)

$$ 
f(x+h) = f(x) + hf'(x) + O(h^2) \\
\frac{f(x+h)-f(x)}{h} = f'(x)+O(h) \\
\therefore \left|\frac{f(x+h)-f(x)}{h} - f'(x)\right| = O(h)
$$

$$
f(x+h) = f(x) + hf'(x) + \frac{1}{2}h^2f''(x) + O(h^3) \\
f(x-h) = f(x) - hf'(x) + \frac{1}{2}h^2f''(x) + O(h^3) \\ 
f(x+h) - f(x-h) = 2hf'(x) + O(h^3) \\ 
\therefore \left|\frac{f(x+h)-f(x-h)}{2h}-f'(x)\right| = O(h^2)
$$

## e)

In [7]:
hw3FD.deltar=0.01
hw3FD.deltat=0.01
hw3FD.useUpwind=False 
(r1, central_diff) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)
hw3FD.useUpwind=True
(r2, upwind) = pricer_bond_vasicek_explicitFD(hw3contract,hw3dynamics,hw3FD)
displayrows=np.logical_and(r1<0.15+hw3FD.deltar/2, r1>0.0-hw3FD.deltar/2)

In [8]:
print(np.stack((r1, central_diff),1)[displayrows])

[[ 1.5000e-01  1.3215e+08]
 [ 1.4000e-01 -1.7579e+07]
 [ 1.3000e-01 -2.0373e+06]
 [ 1.2000e-01  1.4518e+05]
 [ 1.1000e-01  1.1814e+04]
 [ 1.0000e-01 -3.3334e+02]
 [ 9.0000e-02 -9.4642e+00]
 [ 8.0000e-02  7.5960e-01]
 [ 7.0000e-02  7.7241e-01]
 [ 6.0000e-02  7.7459e-01]
 [ 5.0000e-02  7.7720e-01]
 [ 4.0000e-02  7.7966e-01]
 [ 3.0000e-02  7.8265e-01]
 [ 2.0000e-02  9.6594e-01]
 [ 1.0000e-02  5.6606e+01]
 [-3.3307e-16 -1.9217e+04]]


In [9]:
print(np.stack((r2, upwind),1)[displayrows])

[[ 0.15    0.7516]
 [ 0.14    0.7541]
 [ 0.13    0.7566]
 [ 0.12    0.7592]
 [ 0.11    0.7617]
 [ 0.1     0.7643]
 [ 0.09    0.7669]
 [ 0.08    0.7695]
 [ 0.07    0.772 ]
 [ 0.06    0.7746]
 [ 0.05    0.7773]
 [ 0.04    0.7799]
 [ 0.03    0.7825]
 [ 0.02    0.7852]
 [ 0.01    0.7878]
 [-0.      0.7905]]


Clearly, upwind is more accurate with a value of 0.7643 while Central Difference gives a value of -3.33e02 which is far too low. 

# f) 

Less 

Greater

## g)

Given that for $r_0 = 0.10$ we have $B_0 = 0.7643$ we can find the YTM using $0.7643(1+YTM)^5 =1 \rightarrow YTM= 0.0552$

# Question 2)
## a)

$$
E = a(\Delta t)^p + b(\Delta x)^q 
$$ 
$$
\min E \quad s.t \quad \Delta x \Delta t = c
$$

$$
\Delta x = \frac{c}{\Delta t} \rightarrow E = a(\Delta t)^p + bc^q(\Delta t)^{-q}
$$
$$
\frac{dE}{dt} = ap(\Delta t)^{p-1} - bqc^q(\Delta t)^{-q-1} = 0
$$
$$
ap(\Delta t)^{p+q} = bqc^q
$$ 
$$
ap(\Delta t)^p = bq(\Delta x)^q
$$
$$
\frac{ap}{bq}(\Delta t)^p = (\Delta x)^q
$$
$$
\Delta x = h(\Delta t)^{p/q} \quad s.t \quad h = \left(\frac{qp}{bq}\right)^{1/q}
$$

# b)

$$
E = a(\Delta t)^p + b(\Delta x)^q
$$ 
$$
\min E \quad s.t \quad \Delta x \Delta t = c
$$
$$
g(\Delta x, \Delta t) = \Delta x\Delta t
$$

Let's define $(\Delta x^*, \Delta t^*)$ to be the optimal values.

$$
\nabla E(\Delta t^*, \Delta x^*) = \lambda \nabla g(\Delta t^*, \Delta x^*) \\ 
\nabla E(t,x) = (ap\cdot t^{p-1}, bq\cdot x^{q-1}) \quad \nabla g(t,x) = (x,t) \\
ap(\Delta t)^{p-1} = \lambda \Delta x \\ 
ap(\Delta t)^{p} = \lambda \Delta x \Delta t\\
\frac{ap(\Delta t)^p}{c} = \lambda 
$$

$$
\Delta x \Delta t = c \\ 
h(\Delta t)^{p/q}\Delta t = c \\
h(\Delta t)^{(p+q)/q} = c \\
\Delta t = \left(\frac{c}{h}\right)^{q/(p+q)}\\
\lambda = \frac{ab\cdot \left(\frac{c}{h}\right)^{pq/(p+q)}}{c}
$$